In [24]:
import pandas as pd
import numpy as np

age_divided_pop = pd.read_csv('data/municipalities/age_divided_population.csv', delimiter=";")
column_names = ["region", "0-5", "6-15", "16-19", "20-66", "67+"]
age_divided_pop.columns = column_names
age_divided_pop['region_id'] = age_divided_pop.region.apply(lambda x: x.split(" ")[0])
age_divided_pop.region_id = age_divided_pop.region_id.apply(lambda x: x.split("-")[1])
age_divided_pop.drop(columns=["region"], inplace=True)
region_names_id = pd.read_csv("data/municipalities/Kommunenummer_navn_2020.csv", delimiter=",").drop_duplicates()
age_divided_pop.region_id = age_divided_pop.region_id.astype("int64")
df = pd.merge(region_names_id, age_divided_pop, on="region_id", sort=True)
df["population"] = df["0-5"] + df["6-15"] + df["16-19"] + df["20-66"] + df["67+"]
df.head()

,region_id,region,0-5,6-15,16-19,20-66,67+,population
0,301,OSLO,48242,71631,25672,473486,77979,697010
1,1101,EIGERSUND,957,1894,739,8756,2441,14787
2,1103,STAVANGER,10034,17919,6740,90971,18483,144147
3,1106,HAUGESUND,2252,4380,1831,23142,5718,37323
4,1108,SANDNES,6462,11237,4057,49483,9211,80450


In [38]:
pop = df[df.columns[2:7]].to_numpy()

In [40]:
pop.shape[1]

5

In [80]:
c = np.array([[[0.005, 0.02, 0.01, 0.02, 0.08, 0.04, 0.01],
    [0.02,  0.05, 0.04, 0.03, 0.2,  0.1,  0.05],
    [0.01,  0.04, 0.04, 0.02, 0.1,  0.1,  0.05],
    [0.02,  0.03, 0.02, 0.02, 0.1,  0.09, 0.03],
    [0.08,  0.2,  0.1,  0.1,  0.5,  0.4,  0.2 ],
    [0.04,  0.1,  0.1,  0.09, 0.4,  0.3,  0.1 ],
    [0.01,  0.05, 0.05, 0.03, 0.2,  0.1,  0.1 ]],
    [[0.4,   0,    0,     0,   0,    0,    0],
    [0,     0.9,  0.09,  0,   0,    0,    0],
    [0,     0.09, 0.5,   0,   0,    0,    0],
    [0,     0,    0,     1,   0,    0,    0],
    [0,     0,    0,     0,   0,    0,    0],
    [0,     0,    0,     0,   0,    0,    0],
    [0,     0,    0,     0,   0,    0,    0]],
    [[0,     0,    0,     0,   0,    0,    0],
    [0,     0,    0,     0,   0,    0,    0],
    [0,     0,    0.1,   0.2, 0.2,  0.06, 0],
    [0,     0,    0.2,   0.4, 0.4,  0.1,  0],
    [0,     0,    0.2,   0.4, 0.5,  0.1,  0],
    [0,     0,    0.06,  0.1, 0.1,  0.03, 0],
    [0,     0,    0,     0,   0,    0,    0]],
    [[0.06,  0.08, 0.1,   0.1,  0.1,  0.1,  0.03],
    [0.08,  0.1,  0.1,   0.1,  0.2,  0.1,  0.04],
    [0.1,   0.1,  0.2,   0.2,  0.2,  0.2,  0.05],
    [0.1,   0.1,  0.2,   0.2,  0.2,  0.2,  0.05],
    [0.1,   0.2,  0.2,   0.2,  0.3,  0.2,  0.07],
    [0.1,   0.1,  0.2,   0.2,  0.2,  0.2,  0.05],
    [0.03,  0.04, 0.05,  0.05, 0.07, 0.05, 0.01]]])

In [81]:
temp = np.zeros(shape=(4,7,7))
for i, mat in enumerate(c):
    for j, row in enumerate(mat):
        for k, col in enumerate(row):
            if k == 2 or k == 4:
                temp[i][j][k] = (c[i][j][k] + c[i][j][k+1]) / 2
            elif j == 2 or j == 4:
                temp[i][j][k] = (c[i][j][k] + c[i][j+1][k]) / 2
            else:
                temp[i][j][k] = col
final = []
for mat in temp:
    t_mat = np.delete(mat, 5, 0)
    t_mat = np.delete(t_mat, 5, 1)
    t_mat = np.delete(t_mat, 3, 0)
    t_mat = np.delete(t_mat, 3, 1)
    final.append(t_mat)

In [85]:
final

[array([[0.005, 0.02 , 0.015, 0.06 , 0.01 ],
        [0.02 , 0.05 , 0.035, 0.15 , 0.05 ],
        [0.015, 0.035, 0.03 , 0.1  , 0.04 ],
        [0.06 , 0.15 , 0.1  , 0.45 , 0.15 ],
        [0.01 , 0.05 , 0.04 , 0.15 , 0.1  ]]),
 array([[0.4  , 0.   , 0.   , 0.   , 0.   ],
        [0.   , 0.9  , 0.045, 0.   , 0.   ],
        [0.   , 0.045, 0.25 , 0.   , 0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.   ]]),
 array([[0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.15, 0.13, 0.  ],
        [0.  , 0.  , 0.3 , 0.3 , 0.  ],
        [0.  , 0.  , 0.  , 0.  , 0.  ]]),
 array([[0.06, 0.08, 0.1 , 0.1 , 0.03],
        [0.08, 0.1 , 0.1 , 0.15, 0.04],
        [0.1 , 0.1 , 0.2 , 0.2 , 0.05],
        [0.1 , 0.15, 0.2 , 0.25, 0.06],
        [0.03, 0.04, 0.05, 0.06, 0.01]])]